In [1]:
import sys
import os 
os.chdir('..')
sys.path.append("./scripts")
import numpy as np
import config
import torch
import utils
import matplotlib.pyplot as plt
import DANF.blong_dnn as dnn
import opt.goodput_opt as go

def dnn_eval_main():
    """
    dnn_{}e_{}s.pt
    best_dnn_round = {1:5000, 2: 6800, 4: 7000, 8: 6800, 16:5200, 32:6400, 64:3400}
    6600 -2.04114123
    """
    # best_dnn_round = {1:6400, 2: 6400, 4: 6800, 8: 5600, 16:6600, 32:6800, 64:3400}
    # 5600
    best_dnn_round = {1:6400, 2: 6400, 4: 6800, 8: 5600, 16:6600, 32:6800, 64:3400}
    all_bcrs = []
    with open("./output/white_noise_dnn_gain_2.log", 'w') as f: 
        for extf in [1, 2, 4, 8, 16, 32, 64]:
            if extf <= 2:
                snr_range = np.arange(-30, 11, 1)
            else:
                snr_range = np.arange(-30, 1, 1)
            # snr_range = np.arange(1, 6, 1)
            f_name = "white_noise_{}e_test".format(extf)
            model_name = "dnn_uni_cpy_{}e_{}s.pt".format(extf, best_dnn_round[extf])
            bcrs = dnn.eval_all(f_name, model_name, extf, snr_range)
            for i in range(bcrs.shape[0]):
                f.write("extf={}, snr={}, dnn={}, stft={}, pd={}\n".format(extf, snr_range[i], bcrs[i, 0], bcrs[i, 1], bcrs[i, 2]))
                f.flush()
            extfs = np.array([extf for _ in range(bcrs.shape[0])])
            all_bcrs.append(np.hstack([snr_range.reshape([-1, 1]), extfs.reshape([-1, 1]), bcrs]))
    all_bcrs = np.vstack(all_bcrs)
    np.savetxt("./output/white_noise_dnn_gain_2.csv", all_bcrs, delimiter=",")

def dnn_eval_train():
    # best_dnn_round = {1:5000, 2: 6800, 4: 7000, 8: 6800, 16:6400, 32:6400, 64:3400}
    best_dnn_round = {1:6400, 2: 6400, 4: 6800, 8: 5600, 16:6600, 32:6800, 64:3400}
    all_bcrs = []
    with open("./output/white_noise_dnn_gain_train_2.log", 'w') as f: 
        for extf in [1, 2, 4, 8, 16, 32, 64]:
            if extf <= 2:
                snr_range = np.arange(-30, 11, 1)
            else:
                snr_range = np.arange(-30, 1, 1)
            f_name = "white_noise_{}e_train".format(extf)
            model_name = "dnn_uni_cpy_{}e_{}s.pt".format(extf, best_dnn_round[extf])
            bcrs = dnn.eval_all(f_name, model_name, extf, snr_range)
            for i in range(bcrs.shape[0]):
                print("extf={}, snr={}, dnn={}, stft={}, pd={}\n".format(extf, snr_range[i], bcrs[i, 0], bcrs[i, 1], bcrs[i, 2]))
                f.write("extf={}, snr={}, dnn={}, stft={}, pd={}\n".format(extf, snr_range[i], bcrs[i, 0], bcrs[i, 1], bcrs[i, 2]))
                f.flush()
            extfs = np.array([extf for _ in range(bcrs.shape[0])])
            all_bcrs.append(np.hstack([snr_range.reshape([-1, 1]), extfs.reshape([-1, 1]), bcrs]))
    all_bcrs = np.vstack(all_bcrs)
    np.savetxt("./output/white_noise_dnn_gain_train_2.csv", all_bcrs, delimiter=",")

def dnn_wifi_int_eval_main():
    # best_dnn_round = {1:6400, 2: 6400, 4: 6800, 8: 5600, 16:6600, 32:6800, 64:3400}
    best_dnn_round = {1:3800, 2: 5600, 4: 3400, 8: 6600, 16:5800, 32:6600, 64:3200}

    best_dnn_upd_round = {4: 500, 8: 800, 16:300, 32:400, 64:300}
    all_bcrs = []
    with open("./output/wifi_dnn_gain2.log", 'w') as f: 
        for extf in [4, 8, 16, 32, 64]:
            snr_range = np.arange(-30, 1, 1) 
            f_name = "wifi_int_new_{}e_test".format(extf)
            dnn_name = "dnn_uni_cpy_wr_{}e_{}s.pt".format(extf, best_dnn_round[extf])
            dnn_upd_name = "dnn_uni_cpy_wifi_{}e_{}s.pt".format(extf, best_dnn_upd_round[extf])
            bcrs = dnn.eval_wifi(f_name, dnn_name, dnn_upd_name, extf, snr_range)
            for i in range(bcrs.shape[0]):
                f.write("extf={}, snr={}, dnn={}, dnn_upd={}, stft={}, pd={}\n".format(extf, snr_range[i], bcrs[i, 0], bcrs[i, 1], bcrs[i, 2], bcrs[i, 3]))
                f.flush()


def get_snr_threshold(thres=0.1, f_name="white_noise_dnn_gain", num=3, extfs=[1, 2, 4, 8, 16, 32, 64]):
    # extfs = [1, 2, 4, 8, 16, 32, 64]
    snr_thres = np.zeros([len(extfs), num])
    snr_thres[:, :] = 100
    all_extf_bers = []
    for extf_idx in range(len(extfs)):
        extf = extfs[extf_idx]
        with open("./output/" + f_name + ".log", "r") as f:
            lines = f.readlines()
            snrs = []
            bcrs = []
            for line in lines:
                s = line.split(",")
                s_extf = int(s[0][s[0].find("=")+1:])
                if s_extf != extf:
                    continue 
                snr = int(s[1][s[1].find("=")+1:])
                bcr = np.zeros(num, dtype=np.float32)
                for i in range(num):
                    bcr[i] = float(s[2+i][s[2+i].find("=")+1:])
                snrs.append(snr)
                bcrs.append(bcr)
            snrs = np.array(snrs)
            bcrs = np.array(bcrs)
            idx = np.argsort(snrs)
            snrs = snrs[idx]
            bcrs = 1 - bcrs[idx, :]
            for i in range(len(snrs)-1):
                for j in range(num):
                    if snr_thres[extf_idx, j] == 100 and (bcrs[i, j] >= thres and bcrs[i+1, j] <= thres):
                        diff = bcrs[i+1, j] - bcrs[i, j]
                        slope = diff / (snrs[i+1] - snrs[i])
                        snr_diff = (thres - bcrs[i, j]) / slope
                        snr_thres[extf_idx, j] = snrs[i] + snr_diff
            all_extf_bers.append([snrs, bcrs])
    return snr_thres, all_extf_bers

def get_snr_ber_relation():
    dict = {}
    extfs = [1, 2, 4, 8, 16, 32, 64]
    for extf_idx in range(len(extfs)):
        extf = extfs[extf_idx]
        with open("./output/white_noise_dnn_gain.log", "r") as f:
            lines = f.readlines()
            for line in lines:
                s = line.split(",")
                s_extf = int(s[0][s[0].find("=")+1:])
                if s_extf != extf:
                    continue 
                snr = int(s[1][s[1].find("=")+1:])
                bcr = float(s[2][s[2].find("=")+1:])
                if snr not in dict:
                    dict[snr] = {}
                dict[snr][extf] = bcr
    print(dict)
    return dict 

In [3]:
# dnn_eval_main()  # run DNN for BCR over SNR
extfs = [2, 4, 8, 16, 32, 64]
a, bcrs = get_snr_threshold(extfs=extfs, f_name="white_noise_dnn_gain_2")  # get the snr thres from log
print(a[:, 0] - a[:, 1], np.min(a[:, 0] - a[:, 1]), np.max(a[:, 0] - a[:, 1]))
print(a[:, 0] - a[:, 2], np.min(a[:, 0] - a[:, 2]), np.max(a[:, 0] - a[:, 2]))
for i in range(len(bcrs)):
    print(i, bcrs[i][0].shape, bcrs[i][1].shape)
    np.savetxt("output/snr_relation/cpy_extf={}.csv".format(extfs[i]), bcrs[i][1])


[-0.70887385 -1.62491372 -1.69944063 -2.04114123 -2.54063926 -2.31914604] -2.5406392569555987 -0.7088738540795845
[-0.79248276 -1.18999721 -1.90522494 -2.61633391 -3.37077952 -3.24998565] -3.3707795188290692 -0.7924827592217962
0 (41,) (41, 3)
1 (31,) (31, 3)
2 (31,) (31, 3)
3 (31,) (31, 3)
4 (31,) (31, 3)
5 (31,) (31, 3)


In [4]:
# dnn_wifi_int_eval_main()  # run DNN for BCR over WiFi SINR
snr_thres, _ = get_snr_threshold(f_name="wifi_dnn_gain2", num=4, extfs=[4, 8, 16, 32, 64], thres=0.1)
print(snr_thres)
np.savetxt("output/wifi_int_thres.csv", snr_thres, delimiter=",")

[[ -8.92105317  -9.21333308  -8.02739819  -7.92682983]
 [-11.12328673 -11.31578965  -9.95000011  -9.61904768]
 [-13.62499994 -13.82716084 -12.13333222 -11.38666695]
 [-15.6111104  -15.90769298 -14.12986911 -13.36708928]
 [-17.03883525 -17.27956952 -15.45679    -14.47126414]]
